# Проект команды 306 - Проектный семинар - №5 (Пример реализации и работы RTB)

Состав команды:

1. Алиев Хайрутдин Аллилович
2. Зубов Дмитрий Сергеевич
3. Курбанов Иван Сергеевич
4. Лухнев Игорь Дмитриевич
5. Шишков Максим Алексеевич

## TL;DR
---
В данном документе мы иммитируем запрос фронта к RTB, чтобы посмотреть на то, какие данные он выдаст.

---

## Предварительные зависимости
В следующей ячейке будут установлены необходимые библиотеки через `pip`

In [2]:
# Use !pip install -q [your-package]
!pip install -q PyGithub
!pip install -q fsspec
!pip install -q catboost

Следующая ячейка для импорта установленных пакетов в проект

In [43]:
# use aliases for long names
import os
import requests
from github import Github as gh
from getpass import getpass
import re
import pandas as pd
import numpy as np
import base64
from io import StringIO
from pathlib import Path
from PIL import Image
import torch
import torchvision
from tqdm.auto import tqdm
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torchvision import transforms
import os
import random
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pickle as pk
from catboost import CatBoostClassifier
from io import BytesIO
from sklearn.preprocessing import StandardScaler as ss

In [61]:
to_save = [2,5,7,13,15,17,23,29,30,36,37,42,43,50,53,54,56,60,62,63,66,67,72,73,75,77,80,84,86,87,88,91,95,97,98,100,102,104,111,112,115,118,119,120,121,124,128,130,131,135,137,145,147,150,151,152,155,156,159,168,169,171,172,173,174,175,181,183,184,189,190,191,194,199,201,210,213,214,216,217,228,229,230,231,238,239,243,244,245,247,248,249,252,257,258,263,265,267,270,273,274,282,283,284,286,287,290,292,293,294,299,302,309,314,320,322,332,334,335,336,349,352,356,357,359,364,371,372,380,381,383,384,387,388,390,391,393,394,395,397,398,399,406,407,409,410,411,415,422,425,429,434,436,440,443,451,452,455,458,459,462,463,464,465,472,474,475,481,486,488,490,493,501,503,506,513,516,523,525,526,527,530,538,540,541,546,549,554,556,559,562,565,568,569,581,582,588,591,592,595,598,606,607,611,615,616,617,618,621,624,625,632,633,634,635,636,637,641,649,651,656,657,658,659,662,663,667,668,673,676,681,688,694,695,699,700,702,703,706,709,710,711,713,715,716,721,722,724,727,728,730,731,733,735,741,742,743,746,752,754,755,756,759,760,764,768,771,772,782,783,791,796,798,800,801,804,805,806,807,811,813,814,818,819,824,826,829,830,832,833,838,840,844,846,848,850,856,857,858,860,861,862,868,870,873,874,875,876,878,880,883,884,887,888,890,891,893,894,896,898,900,903,905,907,908,913,914,916,920,926,927,933,935,937,938,939,940,942,943,946,949,956,960,961,962,963,967,973,975,985,993,995,1001,1009,1010,1014,1017,1021,1022,1024,1026,1029,1031,1032,1033,1040,1044,1045,1046,1049,1050,1052,1054,1057,1060,1061,1064,1065,1066,1067,1068,1071,1074,1079,1080,1083,1087,1093,1094,1095,1097,1103,1106,1110,1113,1114,1122,1123,1127,1128,1131,1132,1134,1140,1141,1142,1143,1144,1147,1148,1150,1152,1154,1155,1161,1162,1163,1164,1167,1169,1170,1175,1183,1186,1188,1189,1190,1191,1194,1195,1198,1205,1207,1209,1214,1215,1217,1219,1220,1222,1224,1229,1230,1232,1238,1243,1246,1248,1253,1254,1256,1257,1259,1262,1263,1266,1268,1277,1281,1285,1287,1288,1296,1298,1301,1303,1310,1313,1316,1318,1320,1321,1323,1324,1325,1328,1329,1331,1336,1337,1340,1341,1342,1346,1347,1350,1354,1355,1357,1358,1361,1368,1373,1377,1379,1380,1382,1384,1386,1387,1389,1390,1391,1392,1395,1397,1398,1399,1401,1404,1408,1409,1410,1413,1415,1417,1418,1420,1422,1426,1427,1430,1432,1436,1437,1445,1446,1448,1449,1451,1462,1463,1471,1474,1475,1476,1477,1478,1482,1487,1489,1492,1494,1499,1500,1502,1504,1515,1517,1519,1530,1533,1541,1545,1548,1551,1553,1556,1562,1564,1567,1571,1572,1573,1579,1580,1587,1588,1591,1596,1597,1600,1601,1602,1604,1605,1606,1608,1609,1610,1612,1614,1626,1628,1630,1631,1632,1637,1638,1639,1648,1649,1652,1658,1661,1664,1667,1669,1674,1675,1678,1681,1682,1684,1686,1688,1699,1700,1702,1704,1711,1716,1719,1721,1724,1727,1728,1731,1739,1743,1749,1750,1753,1754,1757,1758,1759,1767,1774,1775,1791,1792,1794,1797,1798,1800,1802,1804,1805,1806,1813,1815,1820,1821,1822,1824,1825,1832,1834,1835,1837,1840,1845,1847,1853,1856,1861,1862,1864,1865,1866,1868,1869,1870,1872,1873,1879,1881,1883,1885,1886,1887,1895,1897,1898,1902,1904,1908,1909,1916,1918,1923,1925,1927,1930,1931,1932,1934,1935,1937,1940,1941,1942,1946,1948,1949,1953,1954,1955,1969,1970,1971,1973,1976,1987,1994,1995,1996,1997,1998,2002,2003,2005,2006,2007,2008,2011,2017,2019,2028,2030,2031,2035,2036,2037,2040,2041,2043,2047]

Зафиксируем random_seed в нашем документе

In [62]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
    
set_random_seed(306)

## Загрузка данных 

Запросим у пользователя креды для подключения к репозиторию

In [5]:
login = input('Enter your login: ')
password = getpass('Enter the secret value: ')

Enter your login:  IgorLukhnev
Enter the secret value:  ········································


Подключимся к репозиторию

In [6]:
repo_path = "IgorLukhnev/FarFetchRS"
# Запишем токен в переменную окружения
g_token = os.getenv('GITHUB_TOKEN', password)
# Вополним коннект
g = gh(g_token)
# Подключимся к репо
repo = g.get_repo(repo_path)

Данные будем загружать на основе датасета, подготовленного на этапе генерации классов

In [7]:
dir_path = 'Data/Classes gen'
data_path = 'Data/Classes gen/classified_data.csv'
contents = repo.get_contents(dir_path)
for x in contents:
    if x.path == data_path:
        data = pd.read_csv(StringIO(base64.b64decode(repo.get_git_blob(x.sha).content).decode("utf8")))

data.head()

,0,1,2,3,4,5,6,7,8,9,...,622,623,624,625,626,627,628,629,630,target
0,3.554062,0.355402,10.802802,0.793606,-1.309952,-9.472051,-2.900915,-4.149205,13.885813,5.210749,...,0.002333,-0.001794,0.000331,-0.000239,-0.000049,-0.000247,-0.000173,-0.000079,-1.334758e-06,20
1,-2.494585,-0.255185,1.570479,-4.664831,3.930491,2.727839,2.492876,0.702715,-5.707938,-1.323667,...,0.001508,-0.002773,-0.000022,0.000383,-0.000006,-0.000247,-0.000023,0.000049,5.132146e-06,38
2,-5.380053,-0.262197,5.081867,-3.779809,2.679179,-0.306436,-5.215407,1.240887,-3.398386,-2.992086,...,-0.001190,0.000850,-0.002119,-0.000512,-0.000018,0.000007,0.000148,-0.000005,-6.000842e-06,38
3,13.143531,-0.692318,-4.766358,-0.613166,0.186990,2.299973,-1.817547,-3.073323,5.928221,2.628333,...,-0.000170,-0.002343,-0.000719,0.000229,0.000211,-0.000022,0.000135,0.000065,2.467087e-07,12
4,4.108840,-0.310048,-0.661627,-7.529811,5.816847,5.253184,0.372228,-3.101277,7.285470,0.232564,...,-0.001350,0.001195,-0.006202,0.000097,0.000605,-0.000089,-0.000164,0.000072,7.688773e-06,40


Загрузим сразу все необходимые для работы данного этапа модели

In [8]:
resnet = torchvision.models.resnet50()
resnet.fc = nn.Linear(in_features=2048, out_features=20, bias=True)
resnet.load_state_dict(torch.load('../input/farfetch-rs-models/resnet_on_clothing_tuned', map_location=torch.device('cpu')))

embedding_model = nn.Sequential(*list(resnet.children())[:-1])
embedding_model.eval()

pca = pk.load(open("../input/farfetch-rs-models/pca.pkl",'rb'))

clf = CatBoostClassifier()
clf.load_model("../input/farfetch-rs-models/best_classifier_ever_300_epochs")

## Мок API

Нам нужно эмулировать запрос от API так, как будто бы мы уже имеем подключение к фронту

In [9]:
class API:
    def __init__(self, ffcollection, img, cat, category1, sex, avSizes, Total_Price, Discount, Final_Price):
        self.ffcollection = ffcollection
        self.img_url = img
        self.cat = 'clothing'
        self.catefory1 = 'trousers'
        self.sex = 'm'
        self.avSizes = avSizes
        self.Total_Price = Total_Price
        self.Discount = Discount
        self.Final_Price = Final_Price
        
    def ask_rtb(self):
        return {'request': 
                ';'.join([
                    self.ffcollection, 
                    self.avSizes, 
                    str(self.Total_Price), 
                    str(self.Discount), 
                    str(self.Final_Price), 
                    self.img_url
                ]),
                'k': random.choice(range(1, 15))
               }

Создадим наше API

In [10]:
api = API(
    random.choice(['unknown', 'New Season', 'Exclusive', 'Positively Conscious', 'Final Sale']), 
    'https://cdn-images.farfetch-contents.com/16/34/85/18/16348518_31500351_480.jpg',
    None,
    None,
    None,
    ', '.join(random.sample(['XXS', 'XS', 'S', 'M', 'L', 'XL', 'XXL'], random.choice(range(1, 7)))),
    853.0,
    0.4,
    512.0
)

In [11]:
api.ask_rtb()

{'request': 'Positively Conscious;XXL, XS, S, XL, M;853.0;0.4;512.0;https://cdn-images.farfetch-contents.com/16/34/85/18/16348518_31500351_480.jpg',
 'k': 12}

## Обработка запросов от API

Теперь мы готовы к обработке запросов от API на бэке

In [12]:
transform = transforms.Compose(
    [
     transforms.Resize(224), # меняем размер изображения
     transforms.RandomHorizontalFlip(p=0.5), # половину изображений перевернем по горизонтали
     transforms.RandomAdjustSharpness(sharpness_factor=2), # увеличим где-то резкость
     transforms.RandomAutocontrast(), # применим автоконтраст
     transforms.RandomCrop(200), # часть фотографий обрежем немного
     transforms.Resize(224), # часть фотографий и снова растянем
     transforms.ToTensor(), # преобразуем в тензор
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]) # нормализуем значения пикселей по цветовым слоям
    ]
    )

In [58]:
def process_request(request_dict, transform):
    data = request_dict['request'].split(';')
    img_url = data[-1]
    img_data = requests.get(img_url).content
    img = Image.open(BytesIO(img_data))
    img = img.convert('RGB')
    img = transform(img).unsqueeze(0).to('cpu')
    features = embedding_model(img).squeeze(3).squeeze(2).squeeze(0).detach().to('cpu').numpy()
    
    features = features[to_save]
    
    features = np.append(np.array([float(x) for x in data if x.replace('.', '').isdigit()]), features)
    features = np.append(features, np.array([1 if x == data[0] else 0 for x in ['unknown', 'Sale', 'New Season', 'Exclusive', 'Positively Conscious', 'Final Sale']]))
    features = np.append(features, np.array([1 if re.search(x, data[1]) is not None else 0 for x in ['XXS', 'XS', 'S', 'M', 'L', 'XL', 'XXL']]))
    
    scaler = ss()
    features = scaler.fit_transform(features.reshape(1, -1))
    
    features = pca.transform(features.reshape(1, -1))
    
    cls = clf.predict(features).item()
    
    return cls

In [59]:
cls = process_request(api.ask_rtb(), transform)
mask = data['target'] == cls

to_range = data[mask]

In [60]:
to_range

,0,1,2,3,4,5,6,7,8,9,...,622,623,624,625,626,627,628,629,630,target
75,7.373593,-0.586520,-1.467821,1.128240,-1.626224,-1.483663,-5.263819,-2.421623,5.684820,2.958794,...,0.000462,-0.003458,-0.000554,-0.000377,0.000221,-0.000042,-0.000113,-0.000005,2.352872e-07,44
160,0.778237,-0.729363,-3.566593,0.322051,-0.883368,0.709490,-3.666499,-1.022321,-1.228332,1.098641,...,0.000177,-0.000416,-0.000404,0.000184,-0.000435,-0.000225,0.000060,-0.000028,-2.540596e-06,44
161,3.551517,-0.838396,-7.541941,-3.886791,2.524093,7.240748,-5.009241,-2.784947,5.174994,1.531472,...,0.001138,0.001084,-0.000892,0.000119,-0.000174,-0.000018,0.000014,-0.000039,-1.148560e-06,44
240,1.201645,-0.345872,1.549780,-1.051484,0.118033,-1.782973,-5.083230,-1.477308,6.722564,2.578687,...,0.000055,-0.002416,-0.001599,-0.000027,0.000630,0.000049,-0.000222,0.000004,3.560407e-06,44
255,0.227740,-0.476423,1.173123,0.009431,-0.700256,-2.522612,-4.797233,-1.928285,4.500940,2.935313,...,0.000795,0.001525,-0.000350,-0.000005,0.000167,-0.000039,0.000146,-0.000019,-2.067076e-06,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6268,2.007266,-0.245748,0.946249,-3.223231,2.310528,0.400465,1.846418,-3.545011,5.609665,3.814197,...,-0.000644,0.001651,-0.000683,-0.000227,-0.000838,0.000030,0.000030,0.000084,2.290739e-06,44
6270,1.370822,-0.491371,-0.461946,-2.851129,1.638661,1.377300,-4.044519,-3.211600,7.782393,2.558442,...,-0.001942,0.002775,-0.000777,-0.000169,0.000112,0.000164,0.000049,0.000022,-2.725280e-06,44
6273,2.247274,-0.692373,-2.962416,-3.613920,2.252697,3.503093,-4.428444,-2.509082,6.592979,1.894920,...,0.002283,-0.002305,0.000205,-0.000339,-0.000087,0.000126,-0.000037,-0.000039,-2.185918e-06,44
6308,0.885552,-0.683003,-5.116373,-3.137717,2.005141,4.569493,-1.321141,-1.770924,5.273157,2.957430,...,-0.000284,-0.004494,0.000605,0.000223,0.000232,-0.000024,-0.000013,-0.000029,7.888707e-06,44


## Ранжирование

### Функция оценки релевантности 